# Banking Fraud with various Predictions

## Contents

1. [Background](#Background)
2. [Setup](#Setup)
3. [Data](#Data)
4. [Exploratory Data Analysis](#Exploratory-Data-Analysis)
    1. [Investigating the distribution of fraudulent values](#Investigating-the-distribution-of-fraudulent-values)
    2. [Investigating transaction flows](#Investigating-transaction-flows)
    3. [Investigating account names](#Investigating-account-names)
    4. [Investigating transactions](#Investigating-transactions)
    5. [Investigating time](#Investigating-time)
5. [Feature Engineering](#Feature-Engineering)
   1. [Removing columns and specific types](#Removing-columns-and-specific-types)
   2. [Adding the **hourOfDay** feature](#Adding-the-hourOfDay-feature)
   3. [Changing categorical values](#Changing-categorical-values)
   4. [Splitting and standardising](#Splitting-and-standardising)
6. [Models](#Models)

## Background

There is a lack of public available datasets on financial services and specially in the emerging mobile money transactions domain. Financial datasets are important to many researchers and in particular to us performing research in the domain of fraud detection. Part of the problem is the intrinsically private nature of financial transactions, that leads to no publicly available datasets.

Presented here is a synthetic dataset generated using the simulator called PaySim as an approach to such a problem. PaySim uses aggregated data from the private dataset to generate a synthetic dataset that resembles the normal operation of transactions and injects malicious behaviour to later evaluate the performance of fraud detection methods.

## Setup

Import AWS specific modules and specify S3 data location

In [ ]:
# ! pip install pyarrow
# ! pip install s3fs
# ! pip install seaborn
# import sys
# !{sys.executable} -m pip install --upgrade seaborn
import boto3
from random import seed, sample
import sagemaker
from sagemaker import get_execution_role

import io
import pyarrow.parquet as pq

Import python ML modules

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
%matplotlib inline

import numpy as np
import pandas as pd
import seaborn as sns
sns.set_theme()
sns.set_context("paper")

from sklearn.model_selection import cross_val_predict, cross_val_score, train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_curve, auc, precision_score

from sklearn.ensemble import RandomForestClassifier
# from xgboost.sklearn import XGBClassifier

In [ ]:
role = get_execution_role()
bucket='sagemaker-pmelvin'
parquet_data = 'input-parquet-single/part-00000-28137ef0-c04d-436a-8b20-ae663dbe740b-c000.snappy.parquet'

## Data

Import data and perform initial analysis (parquet from S3)

In [ ]:
%%time
buffer = io.BytesIO()
client = boto3.resource('s3')

object = client.Object(bucket, parquet_data)
object.download_fileobj(buffer)

# df = pd.read_parquet(buffer)
df = pq.read_table(buffer).to_pandas()

In [ ]:
df = df.rename(columns={'nameorig':'nameOrig','oldbalanceorg':'oldBalanceOrig', \
                        'newbalanceorig':'newBalanceOrig', 'namedest':'nameDest', \
                        'oldbalancedest':'oldBalanceDest', 'newbalancedest':'newBalanceDest', \
                        'isfraud':'isFraud', 'isflaggedfraud':'isFlaggedFraud'})

In [ ]:
df.head(n=5)

In [ ]:
df.shape

In [ ]:
df.info()

As we can see from above, the dataset has roughly 6.5 million records, where each record uses 11 attributes (features) to describe the profile of the transactions. The attributes are:
- `step`: maps a unit of time in the real world. In this case 1 step is 1 hour of time
- `type`: CASH-IN, CASH-OUT, DEBIT, PAYMENT and TRANSFER
- `amount`: the amount of the transaction in local currency
- `nameOrig`: the customer who started the transaction
- `oldBalanceOrig`: the initial balance before the transaction
- `newBalanceOrig`: the customer's balance after the transaction
- `nameDest`: the recipient ID of the transaction
- `oldBalanceDest`: the initial recipient balance before the transaction
- `newBalanceDest`: the recipient's balance after the transaction
- `isFraud`: this identifies a fraudulent transaction (1) and non fraudulent (0)
- `isFlaggedFraud`: this identifies a flagged fraudulent transaction

# Exploratory Data Analysis

First we check to see if there are any missing values

In [ ]:
df.isnull().values.any()

## Investigating the distribution of fraudulent values

We know there are five transaction types, lets see how the are distributed

In [ ]:
print(df.type.value_counts())
print()

fig = plt.subplots(figsize=(6,4))
ax = df.type.value_counts().plot(kind='bar', title="Transaction type")
ax.set_ylabel("Number of transactions")
plt.show()

We also know that there are two features that look interesting **isFraud** and **isFlaggedFraud** and from the description _isFraud_ indicates actual fraudulent transactions whereas _isFlaggedFraud_ is when the system prevents the transaction due to some condition

Lets see how many transactions are fraudulent

In [ ]:
fig = plt.subplots(figsize=(6,4))
sns.countplot(x='type', hue='isFraud', data=df, order = df['type'].value_counts().index)
plt.show()

Its clear there are some fraudulent transactions but it is very difficult to see how many as in comparison to the entire dataset the amount is tiny

Therefore, lets find out how many are actual fraud and in what type

In [ ]:
fig = plt.subplots(figsize=(6,4))
ax = df.groupby(['type', 'isFraud']).size().plot(kind='bar')
ax.set_xlabel("(type, isFraud)")
ax.set_ylabel("Number of transactions")

for p in ax.patches:
    ax.annotate(str(format(int(p.get_height()), ',d')), (p.get_x(), p.get_height()*1.01))
plt.show()

From the above it looks like that only **CASH_OUT** and **TRANSFER** have actual fraudulent transactions

Now lets have a look at which transactions are being flagged as fraudulent

In [ ]:
fig = plt.subplots(figsize=(6,4))
sns.countplot(x='type', hue='isFlaggedFraud', data=df, order = df['type'].value_counts().index)
plt.show()

Unlike **isFraud** we cannot even see which are flagged as fraudulent so lets find out which type

In [ ]:
fig = plt.subplots(figsize=(6,4))
ax = df.groupby(['type', 'isFlaggedFraud']).size().plot(kind='bar')
ax.set_xlabel("(type, isFlaggedFraud)")
ax.set_ylabel("Number of transactions")

for p in ax.patches:
    ax.annotate(str(format(int(p.get_height()), ',d')), (p.get_x(), p.get_height()*1.01))
plt.show()

From the above plot it looks like that only **TRANSFER** has any flagged transactions and the amount is only 16 records out of 6.5 million!

Due to the insignificant percentage we can also remove th **isFlaggedFraud** feature as well

So we now know that we can focus on **CASH_OUT** and **TRANSFER** and remove the other types, now let's have a look at the source and direction of the transcactions

## Investigating transaction flows

Just as a recap let's look at some of our data

In [ ]:
df.head(n=5)

We will focus on the **nameOrig** and **nameDest** and see if we can see a pattern

The format for the value has either a **C** or **M** before some digits. However, the letters are not defined, and it would be reasonable to assume that **C** stands for **Customer** and **M** stands for **Merchant**, let's have a look at the relationships between transaction flows between **C** and **M**

We will create a new feature called _direction_ which will correlate to the following:
1. CC - Customer to customer
2. CM - Customer to merchant
3. MC - Merchant to customer
4. MM - Merchant to merchant

In [ ]:
%%time
df_direction = df.copy()
df_direction['direction'] = np.nan

df_direction.loc[df.nameOrig.str.contains('C') & df.nameDest.str.contains('C'),"direction"] = "CC" 
df_direction.loc[df.nameOrig.str.contains('C') & df.nameDest.str.contains('M'),"direction"] = "CM"
df_direction.loc[df.nameOrig.str.contains('M') & df.nameDest.str.contains('C'),"direction"] = "MC"
df_direction.loc[df.nameOrig.str.contains('M') & df.nameDest.str.contains('M'),"direction"] = "MM"

Let's split up the fraudulent and non-fraudlent transactions so we can refer to these later

In [ ]:
fraud = df_direction[df_direction["isFraud"] == 1]
valid = df_direction[df_direction["isFraud"] == 0]

In [ ]:
print("Fraud transactions by direction:\n",fraud.direction.value_counts())
print()
print("Valid transactions by direction:\n",valid.direction.value_counts())

From the above it looks like the only fraudulent transactions occur between customer accounts (CC) so like the type above we can remove the extraneous information when we do feature engineering

Using the new table we can confirm that **CASH_OUT** and **TRANSFER** are the only types with fraudulent activity

## Investigating account names

We now need to figure out if the originator and the destination are the same relating to removing the money using the **CASH_OUT**

First we start by confirming that only the **CASH_OUT** and **TRANSFER** have the fraudulent activities

In [ ]:
cash_in=df.loc[(df.isFraud==1) & (df.type=='CASH_IN')]
cash_out=df.loc[(df.isFraud==1) & (df.type=='CASH_OUT')]
debit=df.loc[(df.isFraud==1) & (df.type=='DEBIT')]
payment=df.loc[(df.isFraud==1) & (df.type=='PAYMENT')]
transfer=df.loc[(df.isFraud==1) & (df.type=='TRANSFER')]

print('Fraudulent transacions in CASH_IN',len(cash_in))
print('Fraudulent transacions in CASH_OUT',len(cash_out))
print('Fraudulent transacions in DEBIT',len(debit))
print('Fraudulent transacions in PAYMENT',len(payment))
print('Fraudulent transacionsin TRANSFER',len(transfer))

Which they are, so that is good :) (well not really!)

Now we need to test whether the account to _cashout_ is actually used to remove the money

In [ ]:
fraud_transfer = fraud[fraud["type"] == "TRANSFER"]
fraud_cashout = fraud[fraud["type"] == "CASH_OUT"]

fraud_transfer.nameDest.isin(fraud_cashout.nameOrig).any()

This returns false, so we now know that the account used to remove the fund was not the same as the one that received the funds

Therefore, we can remove the **nameOrig** and **nameDest** as well

## Investigating transactions

What are the amounts of fraudulent vs legitimate transactions

In [ ]:
pd.set_option('float_format', '{:,.2f}'.format)
print("Description of amounts moved in fraudulent transactions: \n",pd.DataFrame.describe(fraud.amount),"\n")
print("Description of amounts moved moved in legitimate transactions: \n", pd.DataFrame.describe(valid.amount),"\n")

From the above we can clearly see that the maximum amount for a fraudulent transaction is 10 million, with the average of 1.5 million, however there is no real limit for legimate transactions (maximum of 100 million) but the average is much lower, around 200 thousand

Now lets have a look at the balances before and after the transaction

In [ ]:
wrong_originating_bal = sum(df["oldBalanceOrig"] - df["amount"] != df["newBalanceOrig"])
wrong_destination_bal = sum(df["newBalanceDest"] + df["amount"] != df["newBalanceDest"])

print("% of transactions with balance errors in the originating account: ", 100*round(wrong_originating_bal/len(df),2))
print("% of transactions with balance errors in the destination account: ", 100*round(wrong_destination_bal/len(df),2))

So it looks like almost all of the balance/transactions have errors, so let's have a closer look

In [ ]:

amount_given = sum(df["amount"] > df["oldBalanceOrig"])
amount_given = "{:,}".format(amount_given)
print("The number of occurances where the amount given is greater than the amount that is in the originator's account: ", amount_given)

amount_received = sum(df["amount"] > df["newBalanceDest"])
amount_received = "{:,}".format(amount_received)
print("The number of occurances where the amount received is greater than the amount that is in the receiver's account: ", amount_received)


So from the above we can confirm that there are some fundamental erros in the datset as normally it would be impossible to give/receive more than than is availble in your account, therefore we cannot ignore this values but simply be aware of them (and the errors within)

## Investigating time

For the final look at the dataset, we will look at the time or **step**

We know that the **step** relates to 1 hour over a period of thrity days and we need to have a look if there is any correlation to fraudulent transactions over days or hours

Let's start with having a look over the entire month

In [ ]:
sns.histplot(valid, x='step', bins=30, color='seagreen')
plt.xlabel("step - 1 hour")
plt.title("# of valid transactions over a month")
plt.show()

sns.histplot(fraud, x='step', bins=30, color='indianred')
plt.xlabel("step - 1 hour")
plt.title("# of fraudulent transactions over a month")
plt.show()

As we can see from the above:
- for valid transactions, the majority occurs between steps 0 - 60 and 125 - 400
- for fraudulent transactions, there is no real pattern, they occur at all times

Now let's have a look over a week

In [ ]:
days = 7

fraud_days = fraud.step % days
valid_days = valid.step % days

sns.histplot(valid_days, bins=days, color='seagreen')
plt.xlabel("step - 1 day")
plt.title("# of valid transactions over a week")
plt.show()

sns.histplot(fraud_days, bins=days, color='indianred')
plt.xlabel("step - 1 day")
plt.title("# of fraudulent transactions over a week")
plt.show()

From the above there is no indication that fraudulent transactions happen on a particular day, they occur every day

Now let's have a look at hours

In [ ]:
hours = 24

fraud_hours = fraud.step % hours
valid_hours = valid.step % hours

sns.histplot(valid_hours, bins=hours, color='seagreen')
plt.xlabel("step - 1 hour")
plt.title("# of valid transactions over a day")
plt.show()

sns.histplot(fraud_hours, bins=hours, color='indianred')
plt.xlabel("step - 1 hour")
plt.title("# of fraudulent transactions over a day")
plt.show()

So, what can we infer from the above?

It looks like that fraudulent transactions occur all hours of the day but valid transactions do not seem to occur in the steps between 0 - 9

Therefore, it is a good idea to create a new feature based on hours

In [ ]:
df['hourOfDay'] = np.nan
df.hourOfDay = df_direction.step % 24

In [ ]:
df.head(n=5)

# Feature Engineering

First, let's have a recap of which features we can remove from our investigations

From the **type** feature we are only interested in **CASH_OUT** and **TRANSFER**

We also don't need to include **nameOrig** and **nameDest**

**isFlaggedFraud** is neglible so we can also remove that (remember there were only sixteen values)

## Removing columns and specific types

Although we can remove these features using pandas, it is worth using other tools (Glue and Athena) to reduce th dataset and then reload and continue with the feature engineering process

In [ ]:
from IPython.display import Image

![title](img/glue_transform.png)

and I need to add some code to combine the standard frames into one **dynamic_Frame=applymapping1.coalesce(1)**

once I have run another crawler, I use Athena to remove the specific types to only leave **CASH_OUT** and **TRANSFER**

SELECT *\
FROM removed_columns\
WHERE type = 'CASH_OUT' or type = 'TRANSFER';

the new file is then saved to S3 which i can then use directly (as it is a CSV) or convert it parquet using Glue (as before)

So some information on the origanl file and the feature engineered (well basic cleaning so far)

Original CSV file size    : 470.7MB\
Origanal Parquet file size: 274.8MB

Columns removed CSV file size    : 319.2MB\
Columns removed Parquet file size: 158.1MB

Feature **type** cleaned CSV file size    : 187.1MB\
Feature **type** cleaned Parquet file size: 80.1MB

In [ ]:
csv_original      = 'input-data/bb_banking_fraud.csv'
csv_removed_cols  = 'input-data/removed_columns/run-1606989332572-part-r-00000'
csv_removed_types = 'input-data/removed-specific-types/01a89090-e9b6-4b07-8a49-b0244d6dc035.csv'

par_original      = 'input-parquet-single/part-00000-28137ef0-c04d-436a-8b20-ae663dbe740b-c000.snappy.parquet'
par_removed_cols  = 'input-parquet-single/removed-columns/part-00000-c88815d4-0920-4ab0-bb77-b160ab9dbc09-c000.snappy.parquet'
par_removed_types = 'input-parquet-single/removed-specific-types/part-00000-c2ad1434-15f7-4fb0-afb2-54757f53e21b-c000.snappy.parquet'

In [ ]:
# local
csv_original      = 'bb_banking_fraud.csv'
csv_removed_cols  = 'run-1606989332572-part-r-00000'
csv_removed_types = '01a89090-e9b6-4b07-8a49-b0244d6dc035.csv'

# par_original      = 'part-00000-28137ef0-c04d-436a-8b20-ae663dbe740b-c000.snappy.parquet'
# par_removed_cols  = 'part-00000-c88815d4-0920-4ab0-bb77-b160ab9dbc09-c000.snappy.parquet'
# par_removed_types = 'part-00000-c2ad1434-15f7-4fb0-afb2-54757f53e21b-c000.snappy.parquet'

Let's load and quickly double-check

In [ ]:
%%time
df_csv_original = pd.read_csv(csv_original)

In [ ]:
%%time
df_csv_removed_cols = pd.read_csv(csv_removed_cols)

In [ ]:
%%time
df_csv_removed_types = pd.read_csv(csv_removed_types)

In [ ]:
df_csv_original.shape

In [ ]:
df_csv_original.head(n=5)

In [ ]:
df_csv_removed_cols.shape

In [ ]:
df_csv_removed_cols.head(n=5)

In [ ]:
df_csv_removed_types.shape

In [ ]:
df_csv_removed_types.head(n=5)

So in summary, so far, for the CSV files the load time comes down by a factor of about 10 which is great!

Let's fix those column names again

In [ ]:
df_csv_removed_types = df_csv_removed_types.rename(columns={'oldbalanceorg':'oldBalanceOrig', 'newbalanceorig':'newBalanceOrig', 'oldbalanceoest':'oldBalanceDest', 'newbalancedest':'newBalanceDest', 'isfraud':'isFraud'})

In [ ]:
df = df_csv_removed_types
df.head(n=5)

In [ ]:
df.shape

## Adding the **hourOfDay** feature

In [ ]:
df_copy = df.copy()

df['hourOfDay'] = np.nan
df.hourOfDay = df_copy.step % 24

In [ ]:
df.head(n=5)

Or we can also perform the same action in Athena

SELECT *,  MOD(step, 24) timeofday\
from removed_columns 

which returns a CSV file with a new column **timeofday** which is _step % 24_

In [ ]:
csv_modulus       = 'a7aa763d-235b-45d1-bed6-3364ead885bc.csv'

 ## Changing categorical values

Most of the machine learning algorithms can not handle categorical variables unless we convert them to numerical values. Many algorithm’s performances vary based on how Categorical variables are encoded.

Categorical variables can be divided into two categories: Nominal (No particular order) and Ordinal (some ordered).

Since there is no ordering we will use **One-Hot encoding**.\
In this method, we map each category to a vector that contains 1 and 0 denoting the presence or absence of the feature. The number of vectors depends on the number of categories for features. This method produces a lot of columns that slows down the learning significantly if the number of the category is very high for the feature.

pandas has the **get_dummies** function

In [ ]:
df_with_dummies = pd.get_dummies(df, prefix=['type'])

In [ ]:
df_with_dummies.head(n=5)


## Splitting and standardising

To train any machine learning model irrespective what type of dataset is being used you have to **split** the dataset into training data and testing data.

scikit has **train_test_split** function

In [ ]:
# to allow reproducible splitting we define a RandomState and seed
randomstate = 25
seed(10)

In [ ]:
X = df_with_dummies.drop("isFraud",1)
y = df_with_dummies.isFraud
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=randomstate)

In [ ]:
print(f"The X_train shape is {X_train.shape} and the y_train shape is {y_train.shape}")
print(f"The X_test shape is {X_test.shape} and the y_test shape is {y_test.shape}")

The above splits the data but is it the best option?

Possible not, as the split will always have the same data and this can lead to what is described as the **bias-variance-tradeoff**

To combat this we can use **cross-validation**, which is a method of model validation which splits the data in creative ways in order to obtain the better estimates of “real world” model performance, and minimise validation error.

**K-fold** validation is a popular method of cross validation which shuffles the data and splits it into k number of folds (groups). In general K-fold validation is performed by taking one group as the test data set, and the other k-1 groups as the training data, fitting and evaluating a model, and recording the chosen score. This process is then repeated with each fold (group) as the test data and all the scores averaged to obtain a more comprehensive model validation score.

TODO

What do we mean by **standardisation**?

**Standardise** generally means changing the values so that the distribution standard deviation from the mean equals one. It outputs something very close to a normal distribution. Scaling is often implied.

**Scaling** generally means to change the range of the values. The shape of the distribution doesn’t change. Think about how a scale model of a building has the same proportions as the original, just smaller. That’s why we say it is drawn to scale. The range is often set at 0 to 1.

Many machine learning algorithms perform better or converge faster when features are on a relatively similar scale and/or close to normally distributed. **Scaling** and **standardising** can help features arrive in more digestible form for these algorithms.

In [ ]:
scaler = StandardScaler()
# scaler = MinMaxScaler()
scaler.fit(X_train)

In [ ]:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)